# Project Title
### Data Engineering Capstone Project

#### Project Summary
 This purpose of the project is to build a data pipeline and warehouse that will serve as the source of data for analysing the relationship of population density and green house emission on global land temeperatures.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [3]:
# Do all imports and installs here
import pandas as pd

## Scope
Over the years, the emissions of green house gases like C02, Methane, e.t.c has greatly affected the ozone layer, depleting it, resulting in the green house effect, an increase in climatic temperatures or global warning. The effect of this cannot be overstated as seen in defrosting of ice layers, increased wiild fires, water shortage, heat waves and others. This green house gases emissions comes form automobiles, industrial actions like crude oil processing and domestic actions like coal burning. The increase in global temperatures (climate change) has been a great source of concern in this decade with many countries signing pacts in order to reduced human actions that aids global warming. Studies has also shown that an increase in population density has an effect on the surface temperatures.

The aim of the project is to create a data warehouse containing structured data can be used by a data science team to investigate the effect of greeen house gases emissions and population density on climatic temperatures in countries over years. 

In carrying out this project, the following steps are taken:

- Gather Land temperature, Green house Emissions, and Population Density data over a period of years
- Carry out exploratory data analysis of the data
- Create a conceptual data model for saving the data into a warehouse
- Carry out ETL process using Spark and save files locally in Parquet format
- Move data into Redshift data warehouse.
- Build a data pipeline using Apache Airflow to carry automate the two previous steps and carry out quality checks


## Data Acquisition

The data for this project was gotten from four sources. This varied sources is then aggregated at the ETL stage.

### Global land temperatures by cities and countries

Data on Global land temperatures by cities and countries was gotten from an open dataset on [Kaggle](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data). 

The GlobalLandTemperaturesByCity contains seven (7) columns and about eight 8.6 million rows.  The  columns include date in months for each year per city from the year 1743 through to 2013. Other columns are the Average temperature, Average Temperature Uncertianty,City, Country, Latitude, Longitude.

The GlobalLandTemperaturesByCountry contains four (4) columns and about 570 thousand rows . The columns include date, Average Temperature, Average Temperature Uncertainty and Country.

### Green House Emissions

The data og green house emission is gotten from an open dataset on [OurWorldInData](https://ourworldindata.org/greenhouse-gas-emissions). It contains data on data emissions from 1990 to 2016  of 193 countries . It contains about 5200 rows and 4 columns including Entity (country), Code (three letter country iso code), Year and Total GHG emissions including LUCF (CAIT)

### Population Density 
The data on population density for countries from the open dataset in [OurWorldInData](https://ourworldindata.org/world-population-growth).

The world population density data contains the average population density of about 262 countries from 1961 to 2017. It contains 14600 rows and 4 columns including Entity(country), Code (thre letter country iso code), Year and Population Density in Per square kilometer of land.

### Country and Continents
The data on country and continent was collected to map continent to country knowing that the data scientist would like to investigate which continent tend to be more affected by climatic issues. The data is gotten from [datahub](https://datahub.io/JohnSnowLabs/country-and-continent-codes-list). It has information on 254 countries and there respective continent. It contains 261 rows and six columns including Country_Name, Continent_Name and Three_Letter_Country_Code



In [5]:
# Read in the data here
city_temp_df = pd.read_csv("./Data/GlobalLandTemperaturesByCity.csv")
count_temp_df = pd.read_csv("./Data/GlobalLandTemperaturesByCountry.csv")


In [7]:
emis_df = pd.read_csv("./Data/TotalGHGEmissions.csv")
country_df = pd.read_csv("./Data/Country.csv")
population_df = pd.read_csv("./Data/PopulationDensity.csv")

## Data Exploration

### Global Land Temperatures By City

In [9]:
city_temp_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [11]:
num_of_entries = len(city_temp_df.index)
city_temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   dt                             object 
 1   AverageTemperature             float64
 2   AverageTemperatureUncertainty  float64
 3   City                           object 
 4   Country                        object 
 5   Latitude                       object 
 6   Longitude                      object 
dtypes: float64(2), object(5)
memory usage: 459.2+ MB


In [13]:
city_temp_df2 = city_temp_df.dropna(subset=["AverageTemperature"])
no_nan_entries = len(city_temp_df2)
city_temp_df2

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
5,1744-04-01,5.788,3.624,Århus,Denmark,57.05N,10.33E
6,1744-05-01,10.644,1.283,Århus,Denmark,57.05N,10.33E
7,1744-06-01,14.051,1.347,Århus,Denmark,57.05N,10.33E
8,1744-07-01,16.082,1.396,Århus,Denmark,57.05N,10.33E
...,...,...,...,...,...,...,...
8599206,2013-04-01,7.710,0.182,Zwolle,Netherlands,52.24N,5.26E
8599207,2013-05-01,11.464,0.236,Zwolle,Netherlands,52.24N,5.26E
8599208,2013-06-01,15.043,0.261,Zwolle,Netherlands,52.24N,5.26E
8599209,2013-07-01,18.775,0.193,Zwolle,Netherlands,52.24N,5.26E


In [15]:
city_temp_df["Country"].nunique()

159

In [17]:
t=pd.to_datetime(city_temp_df["dt"])
print (t.dt.year.min(), t.dt.year.max())
print("number of missing values:",num_of_entries - no_nan_entries)

1743 2013
number of missing values: 364130


### Summary 

Exploring the data shows that there are about 364130 missing values for average temperatures in the database. Also this missing values may occur in a particular month in a year but not in the other months. Therefore getting the yearly average temperature for a city might not be straight forward.

### Global Land Temperature by Country

In [19]:
count_temp_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


In [21]:
count_temp_df.info()
num_of_entries = len(count_temp_df.index)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577462 entries, 0 to 577461
Data columns (total 4 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   dt                             577462 non-null  object 
 1   AverageTemperature             544811 non-null  float64
 2   AverageTemperatureUncertainty  545550 non-null  float64
 3   Country                        577462 non-null  object 
dtypes: float64(2), object(2)
memory usage: 17.6+ MB


In [23]:
count_temp_df2= count_temp_df.dropna(subset=["AverageTemperature"])
no_nan_entries = len(count_temp_df2.index)
count_temp_df2

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
5,1744-04-01,1.530,4.680,Åland
6,1744-05-01,6.702,1.789,Åland
7,1744-06-01,11.609,1.577,Åland
8,1744-07-01,15.342,1.410,Åland
...,...,...,...,...
577456,2013-04-01,21.142,0.495,Zimbabwe
577457,2013-05-01,19.059,1.022,Zimbabwe
577458,2013-06-01,17.613,0.473,Zimbabwe
577459,2013-07-01,17.000,0.453,Zimbabwe


In [25]:
count_temp_df["Country"].nunique()

243

In [27]:
t=pd.to_datetime(count_temp_df["dt"])
print (t.dt.year.min(), t.dt.year.max())
print("number of missing values:", num_of_entries - no_nan_entries)

1743 2013
number of missing values: 32651


### Summary 

Exploring the data shows that there are about 32651 missing values for average temperatures by country in the database.

### Total Global Emissions 

In [29]:
emis_df.head()

,Entity,Code,Year,Total GHG emissions including LUCF (CAIT)
0,Afghanistan,AFG,1990,15140000.0
1,Afghanistan,AFG,1991,15060000.0
2,Afghanistan,AFG,1992,13600000.0
3,Afghanistan,AFG,1993,13430000.0
4,Afghanistan,AFG,1994,13240000.0


In [31]:
emis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5208 entries, 0 to 5207
Data columns (total 4 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Entity                                     5208 non-null   object 
 1   Code                                       5155 non-null   object 
 2   Year                                       5208 non-null   int64  
 3   Total GHG emissions including LUCF (CAIT)  5208 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 162.9+ KB


In [33]:
emis_df.dropna(subset=["Total GHG emissions including LUCF (CAIT)"])

,Entity,Code,Year,Total GHG emissions including LUCF (CAIT)
0,Afghanistan,AFG,1990,15140000.0
1,Afghanistan,AFG,1991,15060000.0
2,Afghanistan,AFG,1992,13600000.0
3,Afghanistan,AFG,1993,13430000.0
4,Afghanistan,AFG,1994,13240000.0
...,...,...,...,...
5203,Zimbabwe,ZWE,2012,67630000.0
5204,Zimbabwe,ZWE,2013,67550000.0
5205,Zimbabwe,ZWE,2014,66100000.0
5206,Zimbabwe,ZWE,2015,67490000.0


In [35]:
emis_df["Entity"].nunique()

193

In [37]:
print (emis_df.Year.min(), emis_df.Year.max())

1990 2016


#### Summary
All data has emissions informations however there are some countries that do not have codes , since this code is not really neccessary we use the data that way

### Country Data

In [39]:
country_df.head(10)

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0
5,Europe,EU,"Andorra, Principality of",AD,AND,20.0
6,Africa,AF,"Angola, Republic of",AO,AGO,24.0
7,North America,NaN,Antigua and Barbuda,AG,ATG,28.0
8,Europe,EU,"Azerbaijan, Republic of",AZ,AZE,31.0
9,Asia,AS,"Azerbaijan, Republic of",AZ,AZE,31.0


In [41]:
country_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Continent_Name             262 non-null    object 
 1   Continent_Code             219 non-null    object 
 2   Country_Name               262 non-null    object 
 3   Two_Letter_Country_Code    261 non-null    object 
 4   Three_Letter_Country_Code  258 non-null    object 
 5   Country_Number             258 non-null    float64
dtypes: float64(1), object(5)
memory usage: 12.4+ KB


In [43]:
country_df["Country_Name"].nunique()


254

In [45]:
country_df[country_df.duplicated(["Country_Name"])]

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
9,Asia,AS,"Azerbaijan, Republic of",AZ,AZE,31.0
17,Asia,AS,"Armenia, Republic of",AM,ARM,51.0
59,Asia,AS,"Cyprus, Republic of",CY,CYP,196.0
84,Asia,AS,Georgia,GE,GEO,268.0
117,Asia,AS,"Kazakhstan, Republic of",KZ,KAZ,398.0
172,North America,NaN,United States Minor Outlying Islands,UM,UMI,581.0
192,Asia,AS,Russian Federation,RU,RUS,643.0
235,Asia,AS,"Turkey, Republic of",TR,TUR,792.0


#### Summary

Exploring the data on the country shows that some of the 3 of the countries have no Three_Letter_Country_Code, likewise some rows have no Continent_code. Also he is duplicate data as there are 254 unique countires but 262 countries in total in the dataset. The dataframe above shows the duplicate countries.

### Population Density

In [47]:
population_df.head()

,Entity,Code,Year,Population density (people per sq. km of land area)
0,Afghanistan,AFG,1961,14.040934
1,Afghanistan,AFG,1962,14.315271
2,Afghanistan,AFG,1963,14.603367
3,Afghanistan,AFG,1964,14.905739
4,Afghanistan,AFG,1965,15.222887


In [49]:
population_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14600 entries, 0 to 14599
Data columns (total 4 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Entity                                               14600 non-null  object 
 1   Code                                                 11978 non-null  object 
 2   Year                                                 14600 non-null  int64  
 3   Population density (people per sq. km of land area)  14600 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 456.4+ KB


In [51]:
print (population_df.Year.min(), population_df.Year.max())

1961 2017


In [55]:
population_df[population_df.duplicated(["Entity","Code","Year"])]

,Entity,Code,Year,Population density (people per sq. km of land area)


#### Summary
The Population density have 2622 countries that have no three letter country code. but there are no duplicates

### Summary 
From all data we see that there different number of unique values for countries and the range of years differs. For the data scientist to be able to carry out meaningfull artist we take a collection of data at the intersection of both the number of countries and range of years. For this reason we are choosing to build a database with 150 countries with a year range of 1990 - 2013

### Data Cleaning

In [ ]:
### Removing unnceessary Data

In [54]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


ModuleNotFoundError: No module named 'pyspark'

In [11]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.